# * Sales Data

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  DIM_CORP_KPI

In [2]:
''' Execute Summary Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summary DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-09-22, 10:56:38

   -> Summary DataFrame : 1710 rows, 18 columns

TDMDBPR : Disconnected


In [3]:
# ''' Automate Currently Period '''

# # curr_yr = chk_src_df['TM_KEY_YR'].max()
# # prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

# # curr_qtr = chk_src_df['TM_KEY_QTR'].max()
# # prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
# prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift()#.max().astype(int)

# # curr_wk = chk_src_df['TM_KEY_WK'].max()
# # prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
# # last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# # curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# # prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

# prev_mth

In [4]:
def my_metric_group(v_name):

    # Get : Parameter
    name = v_name
    result = ''
    
    if re.search('Gross Adds', name): result = 'Gross Adds'
    elif re.search('Inflow M1', name): result = 'Inflow M1'
    elif re.search('Inflow M2', name): result = 'Inflow M2'
    elif re.search('No of AP 1D', name): result = 'No of AP 1D'
    elif re.search('Amt of AP 1D', name): result = 'Amt of AP 1D'
    elif re.search('No of AP In Month', name): result = 'No of AP In Month'
    elif re.search('Amt of AP In Month', name): result = 'Amt of AP In Month'
    # elif re.search('CConnected : onsumer', name): result = 'Gross Adds - Con - Consumer'
    # elif re.search('Connected : DataService', name): result = 'Gross Adds - Con - DataService'
    # elif re.search('Registration : Consumer', name): result = 'Gross Adds - Reg - Consumer'
    # elif re.search('Registration : DataService', name): result = 'Gross Adds - Reg - DataService'
    else: result = 'Unknown' 
    
    return result

In [5]:
''' Add Columns '''

chk_src_df['CHANNEL_GRP'] = np.where(chk_src_df['METRIC_CD'].str.contains('CS$'), 'ALL', 'GEO')
chk_src_df['LOCATION_GRP'] = np.where(chk_src_df['METRIC_NAME'].str.contains('Location'), 'Customer', 'Sales')
chk_src_df['METRIC_GRP'] = chk_src_df.apply(lambda x: my_metric_group(str(x['METRIC_NAME'])), axis=1)
# tmp_df['MAIN_FLAG'] = np.where((tmp_df['METRIC_CD'].str.contains('[0-9]C$|[0-9]H$|[0-9]MCOM$|[0-9]CUS$')) | (tmp_df['METRIC_NAME'].str.contains('New Revenue|Existing Revenue')), 'N', 'Y')
# chk_src_df.tail()

# chk_src_df.loc[chk_src_df['PRODUCT']=='TOL'].tail()

### KPIs Summary

In [6]:
''' PREPAID Summary : 14 rows, 24 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'PREPAID'
# channel_grp = 'GEO'

prepaid_df = chk_src_df.copy()
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==curr_mth]
prepaid_df = prepaid_df.loc[prepaid_df['PRODUCT']==product]
# prepaid_df = prepaid_df.loc[prepaid_df['CHANNEL_GRP']==channel_grp]

prepaid_df['ALL'] = prepaid_df['P']
prepaid_df['TRUE'] = np.where(prepaid_df['COMP_CD']=='TRUE', prepaid_df['P'], 0)
prepaid_df['DTAC'] = np.where(prepaid_df['COMP_CD']=='DTAC', prepaid_df['P'], 0)
prepaid_df['TMP_NAME'] = prepaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

prepaid_df = prepaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
prepaid_df = prepaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = prepaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    prepaid_df[col] = prepaid_df[col].apply(lambda x: format(x, ',.0f'))

prepaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D,2025-09-22 08:00:41,20250901,20250921,"191,930,086","107,237,788","84,692,298"
1,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D - GEO Channel,2025-09-22 08:00:41,20250901,20250921,"174,595,478","105,428,574","69,166,904"
2,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month,2025-09-22 08:00:41,20250901,20250921,"205,241,584","114,620,160","90,621,424"
3,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month - GEO Channel,2025-09-22 08:00:41,20250901,20250921,"187,659,011","112,775,943","74,883,068"
4,PREPAID,Gross Adds,Prepaid Gross Adds,2025-09-22 08:00:41,20250901,20250921,"943,577","506,822","436,755"
5,PREPAID,Gross Adds,Prepaid Gross Adds (Customer Location),2025-09-22 08:05:49,20250901,20250921,"943,577","506,822","436,755"
6,PREPAID,Gross Adds,Prepaid Gross Adds - GEO Channel,2025-09-22 08:00:41,20250901,20250921,"839,637","504,416","335,221"
7,PREPAID,Inflow M1,Prepaid Inflow M1,2025-09-22 08:00:41,20250901,20250919,"196,805,276","104,504,963","92,300,313"
8,PREPAID,Inflow M1,Prepaid Inflow M1 (Customer Location),2025-09-22 08:05:49,20250901,20250919,"196,805,276","104,504,963","92,300,313"
9,PREPAID,Inflow M1,Prepaid Inflow M1 - GEO Channel,2025-09-22 08:00:41,20250901,20250919,"180,915,480","102,844,087","78,071,394"


In [7]:
''' POSTPAID Summary : 8 rows, 16 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'POSTPAID'
# channel_grp = 'GEO'

postpaid_df = chk_src_df.copy()
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==curr_mth]
postpaid_df = postpaid_df.loc[postpaid_df['PRODUCT']==product]
# postpaid_df = postpaid_df.loc[postpaid_df['CHANNEL_GRP']==channel_grp]

postpaid_df['ALL'] = postpaid_df['P']
postpaid_df['TRUE'] = np.where(postpaid_df['COMP_CD']=='TRUE', postpaid_df['P'], 0)
postpaid_df['DTAC'] = np.where(postpaid_df['COMP_CD']=='DTAC', postpaid_df['P'], 0)
postpaid_df['TMP_NAME'] = postpaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

postpaid_df = postpaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
postpaid_df = postpaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = postpaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    postpaid_df[col] = postpaid_df[col].apply(lambda x: format(x, ',.0f'))

postpaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,POSTPAID,Gross Adds,Postpaid Gross Adds B2B,2025-09-22 08:09:43,20250901,20250921,"16,040","8,299","7,741"
1,POSTPAID,Gross Adds,Postpaid Gross Adds B2B - GEO Channel,2025-09-22 08:09:43,20250901,20250921,"3,018","2,481",537
2,POSTPAID,Gross Adds,Postpaid Gross Adds B2C,2025-09-22 08:09:43,20250901,20250921,"103,004","84,930","18,074"
3,POSTPAID,Gross Adds,Postpaid Gross Adds B2C - GEO Channel,2025-09-22 08:09:43,20250901,20250921,"44,949","38,482","6,467"
4,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B,2025-09-22 08:09:43,20250901,20250921,"6,582,546","3,619,841","2,962,705"
5,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B - GEO Channel,2025-09-22 08:09:43,20250901,20250921,"2,478,707","2,092,323","386,384"
6,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C,2025-09-22 08:09:43,20250901,20250921,"48,026,398","40,782,549","7,243,849"
7,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C - GEO Channel,2025-09-22 08:09:43,20250901,20250921,"23,413,664","19,908,507","3,505,157"


In [8]:
''' TOL Summary : 20 KPIs'''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TOL'
# channel_grp = 'GEO'

tol_df = chk_src_df.copy()
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==curr_mth]
tol_df = tol_df.loc[tol_df['PRODUCT']==product]
# tol_df = tol_df.loc[tol_df['CHANNEL_GRP']==channel_grp]

tol_df = tol_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tol_df = tol_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tol_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tol_df[col] = tol_df[col].apply(lambda x: format(x, ',.0f'))

tol_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TOL,Gross Adds,TOL Gross Adds Connected : Consumer,2025-09-22 08:00:18,20250901,20250921,"32,568"
1,TOL,Gross Adds,TOL Gross Adds Connected : Consumer (Install L...,2025-09-22 08:00:18,20250901,20250921,"32,568"
2,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Channel,2025-09-22 08:00:18,20250901,20250921,"25,213"
3,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Chan...,2025-09-22 08:00:18,20250901,20250921,"25,213"
4,TOL,Gross Adds,TOL Gross Adds Connected : DataService,2025-09-22 08:00:18,20250901,20250921,960
5,TOL,Gross Adds,TOL Gross Adds Connected : DataService (Instal...,2025-09-22 08:00:18,20250901,20250921,960
6,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-09-22 08:00:18,20250903,20250919,40
7,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-09-22 08:00:18,20250903,20250919,40
8,TOL,Gross Adds,TOL Gross Adds Registration : Consumer,2025-09-22 08:00:18,20250901,20250921,"44,934"
9,TOL,Gross Adds,TOL Gross Adds Registration : Consumer - GEO C...,2025-09-22 08:00:18,20250901,20250921,"34,228"


In [9]:
''' TVS Summary : 8 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TVS'
# channel_grp = 'GEO'

tvs_df = chk_src_df.copy()
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==curr_mth]
tvs_df = tvs_df.loc[tvs_df['PRODUCT']==product]
# tvs_df = tvs_df.loc[tvs_df['CHANNEL_GRP']==channel_grp]

tvs_df = tvs_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tvs_df = tvs_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tvs_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tvs_df[col] = tvs_df[col].apply(lambda x: format(x, ',.0f'))

tvs_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TVS,Gross Adds,TVS CMDU Gross Adds,2025-09-22 08:11:10,20250901,20250920,75
1,TVS,Gross Adds,TVS CMDU Gross Adds - GEO Channel,2025-09-22 08:11:10,20250903,20250912,2
2,TVS,Gross Adds,TVS Now Gross Adds,2025-09-22 08:11:10,20250901,20250920,"2,234"
3,TVS,Gross Adds,TVS Now Gross Adds - GEO Channel,2025-09-22 08:11:10,20250901,20250920,"2,067"
4,TVS,Inflow M1,TVS CMDU Inflow M1,2025-09-22 08:11:10,20250901,20250920,"254,519"
5,TVS,Inflow M1,TVS CMDU Inflow M1 - GEO Channel,2025-09-22 08:11:10,20250903,20250912,"3,884"
6,TVS,Inflow M1,TVS Now Inflow M1,2025-09-22 08:11:10,20250901,20250920,"609,221"
7,TVS,Inflow M1,TVS Now Inflow M1 - GEO Channel,2025-09-22 08:11:10,20250901,20250920,"578,584"


### Monitoring

In [10]:
''' Current Month Gross Adds '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH']==curr_mth]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-09-22 08:09:43,20250901,20250916,0,"18,074","17,982","17,982","17,982","17,982","17,982",7,4294,18476
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-09-22 08:09:43,20250901,20250916,0,"7,741","2,674","2,674","2,674","2,674","2,674",7,786,2588
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-09-22 08:09:43,20250901,20250921,0,"84,930","54,708","54,708","54,708","54,708","54,708",7,1255,17801
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-09-22 08:09:43,20250901,20250921,0,"8,299","2,487","2,487","2,487","2,487","2,487",7,590,3015
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-09-22 08:00:41,20250901,20250917,0,"436,755","430,752","430,752","420,443","430,752","429,072",7,6791,57830
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-09-22 08:00:41,20250901,20250921,0,"506,822","387,838","387,838","380,251","387,838","387,714",7,5823,75650
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-09-22 08:00:18,20250901,20250921,0,"32,568","25,210","25,210","24,927","25,210","25,207",7,1135,15174
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-09-22 08:00:18,20250901,20250921,0,960,40,40,40,40,40,7,67,137
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-09-22 08:00:18,20250901,20250921,0,"44,934","34,212","34,212","33,826","34,212","34,208",7,1137,15797
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-09-22 08:00:18,20250901,20250921,0,"1,226",60,60,60,60,60,7,76,164


In [11]:
''' Average Gross Adds per Month '''

avg_range_start = '202401'
avg_range_end = '202503'
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-06-07 14:21:18,20240101,20250331,0,"61,560","59,720","59,720","59,720","59,720","59,720",7,6204,697820
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-06-07 14:21:18,20240101,20250331,0,"10,050","5,366","5,366","5,366","5,366","5,366",7,1528,80500
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-06-07 14:21:18,20240101,20250331,0,"152,588","103,167","103,167","103,167","103,167","103,167",7,2211,438943
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-06-07 14:21:18,20240101,20250331,0,"25,021","8,124","8,124","8,124","8,124","8,124",7,781,64761
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-09-03 14:02:38,20240101,20250331,0,"1,393,899","1,231,493","1,207,608","1,283,526","1,301,464","1,293,565",7,7441,1682801
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-09-03 14:02:38,20240101,20250331,0,"1,159,625","989,901","938,188","918,524","940,606","940,541",7,5822,1787837
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-04-30 14:02:03,20240102,20250331,0,"46,948","33,562","33,562","34,737","35,069","35,055",7,1214,331274
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-04-30 14:02:03,20240102,20250331,0,"1,584",34,34,35,35,35,7,99,2156
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-04-30 14:02:02,20240101,20250331,0,"64,322","45,578","45,578","47,202","47,631","47,608",7,1243,348927
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-04-30 14:02:02,20240101,20250331,0,"3,354","1,033","1,033","1,025","1,043","1,043",7,620,25738


In [12]:
''' Current Month Inflow '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH']==curr_mth]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-09-22 08:09:43,20250901,20250916,0,"7,243,849","7,198,542","7,198,542","7,198,542","7,198,542","7,198,542",7,4294,18476
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-09-22 08:09:43,20250901,20250916,0,"2,962,705","1,176,522","1,176,522","1,176,522","1,176,522","1,176,522",7,786,2588
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-09-22 08:09:43,20250901,20250921,0,"40,782,549","30,270,410","30,270,410","30,270,410","30,270,410","30,270,410",7,1255,17801
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-09-22 08:09:43,20250901,20250921,0,"3,619,841","2,096,417","2,096,417","2,096,417","2,096,417","2,096,417",7,590,3015
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-09-22 08:00:41,20250901,20250917,0,"92,300,313","91,367,734","91,367,734","88,862,068","91,367,734","90,990,666",7,6777,77767
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-09-22 08:00:41,20250901,20250917,0,"38,691,880","38,482,708","38,482,708","38,080,737","38,482,708","38,038,982",7,7073,107058
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-09-22 08:00:41,20250901,20250919,0,"104,504,963","74,286,632","74,286,632","72,538,570","74,286,632","74,278,979",7,5783,84952
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-09-22 08:00:41,20250901,20250919,0,"42,356,294","41,893,020","41,893,020","41,310,840","41,893,020","41,888,257",7,5457,90920
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-09-22 08:00:18,20250901,20250921,0,"17,718,013","13,885,040","13,885,040","13,719,881","13,885,040","13,883,443",7,1135,15174
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-09-22 08:00:18,20250901,20250921,0,"895,440","75,363","75,363","75,363","75,363","75,363",7,67,137


In [13]:
''' Average Inflow per Month '''

avg_range_start = '202401'
avg_range_end = '202503'
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-06-07 14:21:18,20240101,20250331,0,"399,214,352","390,238,209","390,238,209","390,238,209","390,238,209","390,238,209",7,6204,697820
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-06-07 14:21:18,20240101,20250331,0,"146,087,502","27,656,846","27,656,846","27,656,846","27,656,846","27,656,846",7,1528,80500
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-06-07 14:21:18,20240101,20250331,0,"963,708,492","742,500,893","742,500,893","742,500,893","742,500,893","742,500,893",7,2211,438943
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-06-07 14:21:18,20240101,20250331,0,"84,587,995","36,558,027","36,558,027","36,558,027","36,558,027","36,558,027",7,781,64761
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-09-03 14:02:32,20240101,20250331,0,"2,770,052,101","3,438,779,150","3,490,200,512","3,680,100,579","3,738,694,062","3,713,879,698",7,7418,2535837
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-09-03 14:02:32,20240201,20250331,0,"813,718,059","876,573,483","961,133,197","1,056,478,537","1,068,985,422","1,056,809,034",7,7294,2747069
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-09-03 14:02:32,20240101,20250331,0,"2,092,560,585","2,502,279,844","2,535,601,009","2,461,916,507","2,535,601,009","2,535,437,768",7,5799,2096816
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-09-03 14:02:32,20240101,20250331,0,"801,316,591","960,749,056","1,049,549,983","1,030,456,436","1,049,549,983","1,049,435,343",7,5739,2269235
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-04-30 14:02:02,20240102,20250331,0,"388,750,845","283,220,937","283,220,937","293,275,723","296,337,595","296,214,970",7,1214,331274
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-04-30 14:02:02,20240102,20250331,0,"29,369,420","1,591,398","1,591,398","1,627,784","1,630,883","1,630,883",7,99,2156


### All Metric

In [14]:
''' METRIC Summary '''

# src_t_sales_mth_df = chk_src_df.copy()
# src_t_sales_mth_df = src_t_sales_mth_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

# mod_col_list = src_t_sales_mth_df.iloc[:, 6:7].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=10)}')

' METRIC Summary '

In [15]:
''' METRIC Current Month '''

# # v_tm_key_mth = '202502'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
# src_t_sales_mth_df_cols = ['PRODUCT', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
# print(f'\nParameter input...\n')
# print(f'   -> v_tm_key_mth: {curr_mth}')

# src_t_sales_mth_df = chk_src_df[src_t_sales_mth_df_cols].copy()
# src_t_sales_mth_df = src_t_sales_mth_df.loc[src_t_sales_mth_df['TM_KEY_MTH']==curr_mth]
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

# mod_col_list = src_t_sales_mth_df.iloc[:, 7:14].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=17)}')

' METRIC Current Month '

## Actual : Transaction
->  DIM_CORP_KPI

In [16]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101



TDMDBPR : Connected

   -> Execute query... 2025-09-22, 11:01:19

   -> Daily DataFrame : 24084 rows, 12 columns

TDMDBPR : Disconnected


In [17]:
''' METRIC by Day '''

v_tm_key_day = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

src_sale_day_df = chk_src_df.copy()
src_sale_day_df = src_sale_day_df.loc[src_sale_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_sale_day_df = src_sale_day_df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_sale_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_sale_day_df[col] = src_sale_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_sale_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_sale_day_df.to_string(max_cols=12)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_sale_day_df...

   -> DataFrame : 24084 rows, 12 columns


In [18]:
# ''' xB1S000101CS : Prepaid Gross Adds : xxx '''

# v_tm_key_day = 20250401
# v_metric_cd = ['DB1S000101CS', 'TB1S000101CS']
# print(f'\nDB1S000101CS : Prepaid Gross Adds : DTAC\nTB1S000101CS : Prepaid Gross Adds : TMH\n')
# print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

# xB1S000101CS_day_df = chk_src_df.copy()
# xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
# xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['METRIC_CD'].isin(v_metric_cd)]
# xB1S000101CS_day_df = xB1S000101CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

# mod_col_list = xB1S000101CS_day_df.iloc[:, 5:12].columns.tolist()
# for col in mod_col_list:
#     xB1S000101CS_day_df[col] = xB1S000101CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# # xB1S000101CS_day_df
# print(f'\n{xB1S000101CS_day_df.to_string(max_cols=12)}')

In [19]:
''' TB1S000101CS : Prepaid Gross Adds : TMH '''

v_tm_key_day = 20250401
v_metric_cd = 'TB1S000101CS'
print(f'\nTB1S000101CS : Prepaid Gross Adds : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1S000101CS_day_df = chk_src_df.copy()
TB1S000101CS_day_df = TB1S000101CS_day_df.loc[TB1S000101CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1S000101CS_day_df = TB1S000101CS_day_df.loc[TB1S000101CS_day_df['METRIC_CD']==v_metric_cd]
TB1S000101CS_day_df = TB1S000101CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB1S000101CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB1S000101CS_day_df[col] = TB1S000101CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1S000101CS_day_df


TB1S000101CS : Prepaid Gross Adds : TMH

   -> v_metric_cd: TB1S000101CS
   -> v_tm_key_day: 20250401


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250401,2025-09-03 14:02:38,0,0,0,"25,572","24,883","25,572","25,572"
1,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250402,2025-09-03 14:02:38,0,0,0,"25,896","25,142","25,896","25,891"
2,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250403,2025-09-03 14:02:38,0,0,0,"25,569","24,908","25,569","25,567"
3,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250404,2025-09-03 14:02:38,0,0,0,"26,790","26,147","26,790","26,789"
4,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250405,2025-09-03 14:02:38,0,0,0,"24,931","24,353","24,931","24,929"
...,...,...,...,...,...,...,...,...,...,...,...,...
169,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250917,2025-09-22 08:00:41,0,"24,230","18,294","18,294","17,902","18,294","18,290"
170,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250918,2025-09-22 08:00:41,0,"25,584","19,248","19,248","18,845","19,248","19,243"
171,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250919,2025-09-22 08:00:41,0,"25,174","18,879","18,879","18,561","18,879","18,878"
172,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250920,2025-09-22 08:00:41,0,"23,592","17,405","17,405","17,064","17,405","17,398"


In [20]:
''' DB1S000101CS : Prepaid Gross Adds : DTAC '''

v_tm_key_day = 20250401
v_metric_cd = 'DB1S000101CS'
print(f'\nDB1S000101CS : Prepaid Gross Adds : DTAC\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

DB1S000101CS_day_df = chk_src_df.copy()
DB1S000101CS_day_df = DB1S000101CS_day_df.loc[DB1S000101CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
DB1S000101CS_day_df = DB1S000101CS_day_df.loc[DB1S000101CS_day_df['METRIC_CD']==v_metric_cd]
DB1S000101CS_day_df = DB1S000101CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1S000101CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    DB1S000101CS_day_df[col] = DB1S000101CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
DB1S000101CS_day_df


DB1S000101CS : Prepaid Gross Adds : DTAC

   -> v_metric_cd: DB1S000101CS
   -> v_tm_key_day: 20250401


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250401,2025-09-03 14:02:38,0,0,0,"29,401","29,027","29,401","29,201"
1,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250402,2025-09-03 14:02:38,0,0,0,"31,026","30,584","31,026","30,859"
2,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250403,2025-09-03 14:02:38,0,0,0,"31,834","31,371","31,834","31,701"
3,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250404,2025-09-03 14:02:38,0,0,0,"30,238","29,769","30,238","30,099"
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250405,2025-09-03 14:02:38,0,0,0,"28,573","28,144","28,573","28,411"
...,...,...,...,...,...,...,...,...,...,...,...,...
165,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250913,2025-09-22 08:00:41,0,"33,062","32,670","32,670","31,941","32,670","32,567"
166,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250914,2025-09-22 08:00:41,0,"29,611","29,333","29,333","28,884","29,333","29,170"
167,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250915,2025-09-22 08:00:41,0,"27,482","26,974","26,974","26,340","26,974","26,909"
168,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250916,2025-09-22 08:00:41,0,"24,541","24,072","24,072","23,157","24,072","23,978"


In [21]:
# ''' xB2S010100CS : Postpaid Gross Adds B2C : xxx '''

# v_tm_key_day = 20250401
# v_metric_cd = ['DB2S010100CS', 'TB2S010100CS']
# print(f'\nDB2S010100CS : Postpaid Gross Adds : DTAC\nTB2S010100CS : Postpaid Gross Adds : TMH\n')
# print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

# xB2S010100CS_day_df = chk_src_df.copy()
# xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
# xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['METRIC_CD'].isin(v_metric_cd)]
# xB2S010100CS_day_df = xB2S010100CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

# mod_col_list = xB2S010100CS_day_df.iloc[:, 5:12].columns.tolist()
# for col in mod_col_list:
#     xB2S010100CS_day_df[col] = xB2S010100CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# # xB2S010100CS_day_df
# print(f'\n{xB2S010100CS_day_df.to_string(max_cols=12)}')

In [22]:
''' TB2S010100CS : Postpaid Gross Adds B2C : TMH '''

v_tm_key_day = 20250401
v_metric_cd = 'TB2S010100CS'
print(f'\nTB2S010100CS : Postpaid Gross Adds B2C : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB2S010100CS_day_df = chk_src_df.copy()
TB2S010100CS_day_df = TB2S010100CS_day_df.loc[TB2S010100CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB2S010100CS_day_df = TB2S010100CS_day_df.loc[TB2S010100CS_day_df['METRIC_CD']==v_metric_cd]
TB2S010100CS_day_df = TB2S010100CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB2S010100CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB2S010100CS_day_df[col] = TB2S010100CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB2S010100CS_day_df


TB2S010100CS : Postpaid Gross Adds B2C : TMH

   -> v_metric_cd: TB2S010100CS
   -> v_tm_key_day: 20250401


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250401,2025-07-07 14:40:52,0,"5,935","4,519","4,519","4,519","4,519","4,519"
1,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250402,2025-07-07 14:40:52,0,"5,268","3,746","3,746","3,746","3,746","3,746"
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250403,2025-07-07 14:40:52,0,"5,044","3,485","3,485","3,485","3,485","3,485"
3,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250404,2025-07-07 14:40:52,0,"5,261","3,575","3,575","3,575","3,575","3,575"
4,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250405,2025-07-07 14:40:52,0,"5,445","3,882","3,882","3,882","3,882","3,882"
...,...,...,...,...,...,...,...,...,...,...,...,...
169,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250917,2025-09-22 08:09:43,0,"3,430","1,993","1,993","1,993","1,993","1,993"
170,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250918,2025-09-22 08:09:43,0,"3,449","2,163","2,163","2,163","2,163","2,163"
171,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250919,2025-09-22 08:09:43,0,"3,566","2,274","2,274","2,274","2,274","2,274"
172,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250920,2025-09-22 08:09:43,0,"3,601","2,369","2,369","2,369","2,369","2,369"


In [23]:
''' DB2S010100CS : Postpaid Gross Adds B2C : DTAC '''

v_tm_key_day = 20250401
v_metric_cd = 'DB2S010100CS'
print(f'\nDB2S010100CS : Postpaid Gross Adds B2C : DTAC\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

DB2S010100CS_day_df = chk_src_df.copy()
DB2S010100CS_day_df = DB2S010100CS_day_df.loc[DB2S010100CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
DB2S010100CS_day_df = DB2S010100CS_day_df.loc[DB2S010100CS_day_df['METRIC_CD']==v_metric_cd]
DB2S010100CS_day_df = DB2S010100CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2S010100CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    DB2S010100CS_day_df[col] = DB2S010100CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
DB2S010100CS_day_df


DB2S010100CS : Postpaid Gross Adds B2C : DTAC

   -> v_metric_cd: DB2S010100CS
   -> v_tm_key_day: 20250401


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250401,2025-07-07 14:40:52,0,"1,274","1,267","1,267","1,267","1,267","1,267"
1,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250402,2025-07-07 14:40:52,0,"1,395","1,386","1,386","1,386","1,386","1,386"
2,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250403,2025-07-07 14:40:52,0,"2,263","2,242","2,242","2,242","2,242","2,242"
3,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250404,2025-07-07 14:40:52,0,"2,199","2,170","2,170","2,170","2,170","2,170"
4,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250405,2025-07-07 14:40:52,0,"1,785","1,777","1,777","1,777","1,777","1,777"
...,...,...,...,...,...,...,...,...,...,...,...,...
164,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250912,2025-09-22 08:09:43,0,"1,340","1,333","1,333","1,333","1,333","1,333"
165,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250913,2025-09-22 08:09:43,0,988,982,982,982,982,982
166,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250914,2025-09-22 08:09:43,0,696,693,693,693,693,693
167,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250915,2025-09-22 08:09:43,0,"1,092","1,080","1,080","1,080","1,080","1,080"


In [24]:
''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

v_tm_key_day = 20250601
v_metric_cd = 'TB3S000102CS'
print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3S000102CS_day_df = chk_src_df.copy()
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['METRIC_CD']==v_metric_cd]
TB3S000102CS_day_df = TB3S000102CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3S000102CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3S000102CS_day_df[col] = TB3S000102CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3S000102CS_day_df


TB3S000102CS : TOL Gross Adds Connected : Consumer

   -> v_metric_cd: TB3S000102CS
   -> v_tm_key_day: 20250601


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250601,2025-07-31 14:04:31,0,"1,668","1,238","1,238","1,219","1,238","1,238"
1,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250602,2025-07-31 14:04:31,0,"1,695","1,342","1,342","1,327","1,342","1,342"
2,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250603,2025-07-31 14:04:31,0,"1,732","1,389","1,389","1,376","1,389","1,389"
3,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250604,2025-07-31 14:04:31,0,"1,822","1,461","1,461","1,449","1,461","1,460"
4,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250605,2025-07-31 14:04:31,0,"1,795","1,419","1,419","1,413","1,419","1,419"
...,...,...,...,...,...,...,...,...,...,...,...,...
108,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250917,2025-09-22 08:00:18,0,"1,492","1,184","1,184","1,172","1,184","1,184"
109,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250918,2025-09-22 08:00:18,0,"1,353","1,087","1,087","1,067","1,087","1,087"
110,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250919,2025-09-22 08:00:18,0,"1,368","1,088","1,088","1,079","1,088","1,088"
111,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250920,2025-09-22 08:00:18,0,"1,641","1,192","1,192","1,174","1,192","1,192"


In [25]:
''' TB4S001400CS : TVS Now Gross Adds '''

v_tm_key_day = 20250401
v_metric_cd = 'TB4S001400CS'
print(f'\nTB4S001400CS : TVS Now Gross Adds\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4S001400CS_day_df = chk_src_df.copy()
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['METRIC_CD']==v_metric_cd]
TB4S001400CS_day_df = TB4S001400CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4S001400CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4S001400CS_day_df[col] = TB4S001400CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4S001400CS_day_df


TB4S001400CS : TVS Now Gross Adds

   -> v_metric_cd: TB4S001400CS
   -> v_tm_key_day: 20250401


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TVS,TB4S001400CS,TVS Now Gross Adds,20250401,2025-07-07 14:58:38,0,28,28,28,28,28,28
1,TVS,TB4S001400CS,TVS Now Gross Adds,20250402,2025-07-07 14:58:38,0,30,30,30,30,30,30
2,TVS,TB4S001400CS,TVS Now Gross Adds,20250403,2025-07-07 14:58:38,0,37,37,37,37,37,37
3,TVS,TB4S001400CS,TVS Now Gross Adds,20250404,2025-07-07 14:58:38,0,59,59,59,59,59,59
4,TVS,TB4S001400CS,TVS Now Gross Adds,20250405,2025-07-07 14:58:38,0,61,61,61,61,61,61
...,...,...,...,...,...,...,...,...,...,...,...,...
168,TVS,TB4S001400CS,TVS Now Gross Adds,20250916,2025-09-22 08:11:10,0,140,116,116,116,116,116
169,TVS,TB4S001400CS,TVS Now Gross Adds,20250917,2025-09-22 08:11:10,0,172,157,157,157,157,157
170,TVS,TB4S001400CS,TVS Now Gross Adds,20250918,2025-09-22 08:11:10,0,107,93,93,93,93,93
171,TVS,TB4S001400CS,TVS Now Gross Adds,20250919,2025-09-22 08:11:10,0,59,46,46,46,46,46


## Actual : Issue
->  DIM_CORP_KPI

In [26]:
# ''' Execute Issue by case '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# # v_start_date = 20250101
# print(f'\nParameter input...\n')
# # print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ T.TM_KEY_DAY AS TM_KEY_MTH
#         , T.METRIC_CD, T.METRIC_NAME, T.AREA_TYPE, T.AREA_CD, T.AREA_DESC
#         , T.METRIC_VALUE AS TARGET
#         , A.ACTUAL
#         , O.ORGID_P, O.ZONE_TYPE, O.ORGID_G, O.TDS_SGMD, O.ORGID_H, O.HOP_HINT, O.ORGID_HH, O.D_CLUSTER
        
#     FROM CDSAPPO.DIM_CORP_TARGET T

#     LEFT JOIN (
#         SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
#             , PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
#             , SUM(METRIC_VALUE) AS ACTUAL, MAX(LOAD_DATE) LOAD_DATE
#         FROM CDSAPPO.DIM_CORP_KPI
#         WHERE METRIC_CD = 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
#         AND AREA_TYPE = 'CCAA'
#         AND TM_KEY_DAY >= 20250101
#         GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
#     ) A
#         ON A.METRIC_CD = T.METRIC_CD
#         AND A.AREA_CD = T.AREA_CD
#         AND A.TM_KEY_MTH = T.TM_KEY_DAY
        
#     LEFT JOIN (
#         SELECT DISTINCT ORGID_P, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_H, HOP_HINT, ORGID_HH, D_CLUSTER
#             , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG 
#             , SUBSTR(CCAATT,1,4) AS CCAA, DISTRICT_EN
#         FROM CDSAPPO.DIM_MOOC_AREA NOLOCK
#         WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
#     ) O
#         ON O.CCAA = T.AREA_CD

#     WHERE T.METRIC_CD = 'TB3S000102CS'
#     AND T.AREA_TYPE = 'CCAA'
#     AND T.TM_KEY_DAY >= 202501

#     --ORDER BY T.TM_KEY_DAY, T.METRIC_CD, T.AREA_TYPE, O.ZONE_TYPE, O.ORGID_G, O.ORGID_H, O.ORGID_HH
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Summary DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Issue DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [27]:
# ''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

# v_tm_key_mth = 202501
# # v_metric_cd = 'TB3S000102CS'
# print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
# # print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

# TB3S000102CS_issue_df = chk_src_df.copy()
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['TM_KEY_MTH']==v_tm_key_mth]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'] > 0]
# # TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'].isna()]
# # TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['METRIC_CD']==v_metric_cd]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)

# mod_col_list = TB3S000102CS_issue_df.iloc[:, 6:8].columns.tolist()
# for col in mod_col_list:
#     TB3S000102CS_issue_df[col] = TB3S000102CS_issue_df[col].apply(lambda x: format(x, ',.0f'))
    
# # TB3S000102CS_issue_df
# print(f'\n{TB3S000102CS_issue_df.to_string(max_cols=100)}')

## Output File
->  DIM_CORP_KPI

In [28]:
# ''' Execute Sales Monthly(Backup) '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20240101
# print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ 
#         TM_KEY_YR, TM_KEY_MTH
#         , 'Y2025 v.1' AS VER_FLAG
#         , PRODUCT_GRP AS PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
#         , NULL AS REMARK, NULL AS UPD_TM, NULL AS PPN_TM
#         , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P_ACTUAL
#         , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
#         , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
#         , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
#         , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL_SNAP END) CCAA
#         , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL_SNAP END) CCAATT
#         , SUM(CASE WHEN AREA_CD = '016' THEN ACTUAL_SNAP END) "016"
#         , SUM(CASE WHEN AREA_CD = '040' THEN ACTUAL_SNAP END) "040"
#         , SUM(CASE WHEN AREA_CD = '080' THEN ACTUAL_SNAP END) "080"
#         , MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY, MAX(PPN_TM) LOAD_DATE
#     FROM GEOSPCAPPO.AGG_PERF_NEWCO_CN_BK_2025_01_05
#     WHERE TM_KEY_DAY >= {v_start_date}
#     GROUP BY TM_KEY_YR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
#     --ORDER BY 2, 3, PRODUCT_GRP, COMP_CD, METRIC_CD 
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Sales Monthly DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     monthly_sales_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Sales Monthly DataFrame : {monthly_sales_df.shape[0]} rows, {monthly_sales_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [29]:
''' Execute Sales Monthly '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        SUBSTR(TM_KEY_DAY,1,4) AS TM_KEY_YR, SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'CS$|CG$') THEN 'Y2025' ELSE 'Y2024' END VER_FLAG
        , PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P_ACTUAL
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) "016"
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) "040"
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) "080"
        , MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY, MAX(LOAD_DATE) LOAD_DATE
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE METRIC_CD NOT LIKE 'Error%'
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY SUBSTR(TM_KEY_DAY,1,4), SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
    --ORDER BY 2, 3, PRODUCT, COMP_CD, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Sales Monthly DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    monthly_sales_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Sales Monthly DataFrame : {monthly_sales_df.shape[0]} rows, {monthly_sales_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101



TDMDBPR : Connected

   -> Execute query... 2025-09-22, 11:02:35

   -> Sales Monthly DataFrame : 1710 rows, 22 columns

TDMDBPR : Disconnected


In [30]:
''' Generate CSV file '''

op_dir = 'data'
op_monthly_file = f'sales_monthly_{str_curr_dt}'

monthly_sales_df.to_csv(f'{op_dir}/tmp_{op_monthly_file}.csv', index=False, encoding='utf-8')
print(f'\n   -> Generate "tmp_{op_monthly_file}.csv" successfully')

monthly_sales_df.tail(3)


   -> Generate "tmp_sales_monthly_20250922.csv" successfully


,TM_KEY_YR,TM_KEY_MTH,VER_FLAG,PRODUCT,COMP_CD,METRIC_CD,METRIC_NAME,REMARK,UPD_TM,PPN_TM,...,H,HH,CCAA,CCAATT,016,040,080,START_DAY,END_DAY,LOAD_DATE
1707,2024,202412,Y2025,POSTPAID,TRUE,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,KPI Metric 2025,20250307,20250307,...,2483167.99,2483167.99,2483167.99,2483167.99,None,None,67575.0,20241201,20241231,2025-03-07 14:23:14
1708,2025,202502,Y2025,TVS,TRUE,TB4S001004CS,TVS CMDU Gross Adds,KPI Metric 2025,20250507,20250507,...,126.00,126.00,126.00,126.00,None,None,NaN,20250201,20250228,2025-05-07 14:24:37
1709,2025,202504,Y2025,POSTPAID,DTAC,DB2R010500CG,Postpaid Inflow M1 B2C : DTAC - GEO Channel,KPI Metric 2025,20250707,20250707,...,7691524.67,7691524.67,7691524.67,7691524.67,None,None,259669.0,20250401,20250430,2025-07-07 14:40:52
